In [1]:
include("./trajopt/utils.jl")
include("./trajopt/dynamics.jl")

forward (generic function with 4 methods)

In [2]:
# load nominal trajectory
using JLD2, FileIO
@load "./data/quadstar_N15_traj" my_dict
xnom = my_dict["x"]
unom = my_dict["u"]
tnom = my_dict["t"];
N = size(xnom,2) - 1
dtnom = zeros(N)
for i in 1:N
    dtnom[i] = tnom[i+1]-tnom[i]
end

In [3]:
ix = 12
iu = 4

4

In [4]:
dynamics = QuadrotorDynamics()

QuadrotorDynamics(12, 4, 9, 7, [0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 1.0; 0.0 0.0 … 0.0 0.0], [0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0; … ; 0.0 0.0 0.0 0.0; 1.0 0.0 0.0 0.0], [0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 1.0 0.0; 0.0 0.0 … 0.0 1.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], 1.325, 9.81, 0.03843, 0.02719, 0.060528)

In [5]:
function diff_numeric_central(model::Dynamics,x::Vector,u::Vector)
    ix = length(x)
    iu = length(u)
    eps_x = Diagonal{Float64}(I, ix)
    eps_u = Diagonal{Float64}(I, iu)
    fx = zeros(ix,ix)
    fu = zeros(ix,iu)
    h = 2^(-18)
    for i in 1:ix
        fx[:,i] = (forward(model,x+h*eps_x[:,i],u) - forward(model,x-h*eps_x[:,i],u)) / (2*h)
    end
    for i in 1:iu
        fu[:,i] = (forward(model,x,u+h*eps_u[:,i]) - forward(model,x,u-h*eps_u[:,i])) / (2*h)
    end
    return fx,fu
end

diff_numeric_central (generic function with 1 method)

In [11]:
idx = 5
A,B = diff(dynamics,xnom[:,idx],unom[:,idx])
A_,B_ = diff_numeric_central(dynamics,xnom[:,idx],unom[:,idx])

([0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 -0.1451005770941265; 0.0 0.0 … -0.033153956654132344 0.0], [0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0; … ; 0.0 0.0 36.77822728944011 0.0; 0.0 0.0 0.0 16.521279407883412])

In [12]:
sum(abs.(A - A_))

5.672275010804029e-10

In [13]:
sum(abs.(B - B_))

4.9447085315179606e-11